# It works, we need to add a dvergence free constraint!!!

In [1]:
from ngsolve import *
from netgen.occ import *
from ngsolve.webgui import Draw

In [2]:
shape = Box((-1,-1,-1), (1,1,1)) # create a box
geo = OCCGeometry(shape) # create a geometry from the box
mesh = Mesh(geo.GenerateMesh(maxh=0.2)) # create a mesh from the geometry
Draw (mesh); # draw the mesh

WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 1, 'order3d': 1, 'draw_vol': None…

In [3]:
order = 2
Hcc = HCurlCurl(mesh, order=order) # for the gamma variable
Hdd = HDivDiv(mesh, order=order) # , orderinner=order+1) fr t
# fesalpha = H1(mesh, order=order+1) # , dirichlet=".*")
Hc = HCurl(mesh, order=order) # for the div of a Hdd field


k , dk= Hdd.TnT() 
p, dp = Hc.TnT()
g, dg = Hcc.TnT()

In [4]:
n = specialcf.normal(3)
t = specialcf.tangential(3, True)
bbndtang  = specialcf.EdgeFaceTangentialVectors(3)
tef1 = bbndtang[:,0]
tef2 = bbndtang[:,1]
nu1 = Cross(tef1,t)
nu2 = Cross(tef2,t)

def IncOp(g, dg):
    return InnerProduct(curl(g), curl(dg).trans)*dx \
        +(curl(g)*n) * Cross (dg*n, n) * dx(element_vb=BND) \
        +(curl(dg)*n) * Cross (g*n, n) * dx(element_vb=BND) \
        +(g[nu1,t]*dg[t,tef1]-g[nu2,t]*dg[t,tef2])*dx(element_vb=BBND)

# print ( (gamma*nu1*t)*(dgamma*t*tef1))

def J(g): return g - 0.5*Trace(g)*Id(3)

def Constraint(k, p):
    return InnerProduct(k, Grad(p))*dx - (k*n*n) * (p*n) * dx(element_vb=BND)

In [5]:

MINC = BilinearForm(IncOp(g,dg))

Mcc = BilinearForm(InnerProduct(g, dg)*dx)
Mdd = BilinearForm(InnerProduct(k, dk)*dx)

Mccdd = BilinearForm(trialspace=Hcc, testspace=Hdd)
Mccdd += InnerProduct(g, dk)*dx

MJdd = BilinearForm(trialspace=Hdd, testspace=Hdd)
MJdd += InnerProduct(k, dk)*dx-0.5*Trace(k)*Trace(dk)*dx

with TaskManager():
    MINC.Assemble()
    Mcc.Assemble()
    Mdd.Assemble()
    Mccdd.Assemble()
    MJdd.Assemble()



In [6]:

fes = Hdd*Hc
k, p = fes.TrialFunction()
dk, dp = fes.TestFunction()

PROJ = BilinearForm(fes)
PROJ += InnerProduct(k, dk)*dx
PROJ += Constraint(k, dp)
PROJ += Constraint(dk, p)
PROJ += curl(p)*curl(dp)*dx- 1e-6*p*dp*dx # I don't know why this is needed, 

MASS = BilinearForm(fes)
MASS += InnerProduct(k, dk)*dx
with TaskManager():
    PROJ.Assemble() 
    MASS.Assemble()

In [7]:
# 3 minutes for a cube of size 2 and a mesh of size 0.2 and order 3
with TaskManager():
    inv_PROJ = PROJ.mat.Inverse(inverse="sparsecholesky", freedofs=fes.FreeDofs())
    inv_Mcc = Mcc.mat.Inverse(inverse="sparsecholesky", freedofs=fes.FreeDofs())
    inv_Mdd = Mdd.mat.Inverse(inverse="sparsecholesky", freedofs=fes.FreeDofs())

In [8]:
gf_g = GridFunction(Hcc)
gf_inc_g = GridFunction(Hcc)
peak = exp(-20*(x*x+y*y+z*z))
gf_g.Set( CF((peak,0,0, 0,peak,0,  0,0,peak), dims=(3,3)), dual=True, bonus_intorder=12)

gf_k = GridFunction(Hdd)
gf_J_k = GridFunction(Hdd)





In [9]:
print ("gamma\n")
scene1 = Draw (gf_g, mesh, draw_surf=False, clipping={ "z" : -1 })
#print ("inc gamma\n")
#scene2 = Draw (gf_inc_g[0,0], mesh, draw_surf=False, clipping={ "z" : -1 })


print ("kappa \n")
scene3 = Draw (gf_k, mesh, draw_surf=False, clipping={ "z" : -1 })
#print ("J kappa\n")
#scene4 = Draw (gf_J_k[1,1], mesh, draw_surf=False, clipping={ "z" : -1 })

gamma



WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

kappa 



WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

# book yoshida symplectic
https://en.wikipedia.org/wiki/Leapfrog_integration#cite_note-Yoshida1990-6

    def YoshidaIntegratorStep(tau,u,Mu,v,Mv):
        w0 = - 2**-(1/3)/(2-2**-(1/3))
        w1 = 1/(2-2**-(1/3))

        c= [w0/2 , (w0+w1)/2  , (w0+w1)/2 , w0/2 ]
        d= [w1,w0,w1]

        u.vec.data += c[0]*tau*Mv.mat*v.vec
        v.vec.data += d[0]*tau*Mu.mat*u.vec

        u.vec.data += c[1]*tau*Mv.mat*v.vec
        v.vec.data += d[1]*tau*Mu.mat*u.vec

        u.vec.data += c[2]*tau*Mv.mat*v.vec
        v.vec.data += d[2]*tau*Mu.mat*u.vec 

        u.vec.data += c[3]*tau*Mv.mat*v.vec
        return u,
        
with
    
    gf_g.vec.data += dt*inv_Mcc @ Mccdd.mat.T @ inv_Mdd @ MJdd.mat * gf_k.vec

In [10]:
w0 = - 2**-(1/3)/(2-2**-(1/3))
w1 = 1/(2-2**-(1/3))

c= [w0/2 , (w0+w1)/2  , (w0+w1)/2 , w0/2 ]
d= [w1,w0,w1]

MKK= inv_Mcc @ Mccdd.mat.T @ inv_Mdd @ MJdd.mat 
MGG = inv_Mdd @ Mccdd.mat @ inv_Mcc @ MINC.mat

tau = 0.2e-2
final_time = 0.4
with TaskManager():
  for i in range(round(final_time/tau)):
    NORM = Norm(gf_g.vec)
    if NORM > 1.0:
      break

    print ("t = ", i*tau, "norm = ",NORM)

    
    #gf_J_k.vec.data = inv_Mdd @ MJdd.mat * gf_k.vec 
    
    gf_k.vec.data += c[0]*tau*MGG*gf_g.vec
    gf_g.vec.data += d[0]*tau*MKK*gf_k.vec

    
    gf_k.vec.data += c[1]*tau*MGG*gf_g.vec
    gf_g.vec.data += d[1]*tau*MKK*gf_k.vec

    
    gf_k.vec.data += c[2]*tau*MGG*gf_g.vec 
    gf_g.vec.data += d[2]*tau*MKK*gf_k.vec

    gf_k.vec.data += c[3]*tau*MGG*gf_g.vec

    if i % 10 == 0:
        scene1.Redraw()
        #scene2.Redraw()
        scene3.Redraw()
        #scene4.Redraw()
        # scene5.Redraw()


t =  0.0 norm =  0.6165032284539174
t =  0.002 norm =  0.6164824669707907
t =  0.004 norm =  0.6164201893705198
t =  0.006 norm =  0.6163164162126473
t =  0.008 norm =  0.6161711818038675
t =  0.01 norm =  0.6159845342594313
t =  0.012 norm =  0.6157565355895319
t =  0.014 norm =  0.6154872618110931
t =  0.016 norm =  0.6151768030854999
t =  0.018000000000000002 norm =  0.6148252638829081
t =  0.02 norm =  0.6144327631738964
t =  0.022 norm =  0.6139994346492972
t =  0.024 norm =  0.613525426969177
t =  0.026000000000000002 norm =  0.6130109040420015
t =  0.028 norm =  0.6124560453351335
t =  0.03 norm =  0.6118610462178914
t =  0.032 norm =  0.6112261183384943
t =  0.034 norm =  0.6105514900363149
t =  0.036000000000000004 norm =  0.6098374067909562
t =  0.038 norm =  0.609084131709772
t =  0.04 norm =  0.6082919460555745
t =  0.042 norm =  0.6074611498163952
t =  0.044 norm =  0.606592062319303
t =  0.046 norm =  0.6056850228904559
t =  0.048 norm =  0.6047403915637424
t =  0.05 norm

In [11]:
PEAK = CF((peak,0,0, 0,peak,0,  0,0,peak), dims=(3,3))
Draw (gf_g - PEAK, mesh, draw_surf=False, clipping={ "z" : -1 })

WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

BaseWebGuiScene

In [12]:
w0 = - 2**-(1/3)/(2-2**-(1/3))
w1 = 1/(2-2**-(1/3))

c= [w0/2 , (w0+w1)/2  , (w0+w1)/2 , w0/2 ]
d= [w1,w0,w1]

print(c)
print(d)

[-0.3289815435887515, 0.08550922820562423, 0.08550922820562423, -0.3289815435887515]
[0.8289815435887514, -0.657963087177503, 0.8289815435887514]
